In [1]:
%load_ext autoreload
%autoreload 2 

import os
import numpy as np
import pickle

from tqdm import tqdm

import torch
import torch.utils.data
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

from collections import OrderedDict

import nltk
from nltk.tokenize import word_tokenize

from embeddings import load_embeddings, load_vocab
from load_conll import load_conll03
from loader import prepare_sentence, tag_mapping, cap_feature, CoNLLDataset, pad_list, CoNLLDataset_chars
from loader import loader, pad_chars
from model_char5 import Tagger
from torch_utils import prepare_sequence, prepare_sequence_float, tensor
from utils import sent2seq, sent2chars, word_index, char_index, add_unknown_last, zero_digits, save_mappings, load_mappings, reverse_dict
from eval import eval, micro_precision_recall_f1_accuracy, eval_metrics, eval_metrics_crf, save_plot

Using TensorFlow backend.


In [2]:
#Parameters
torch.manual_seed(0)

parameters = OrderedDict()

parameters["lr"] = 0.001
parameters["optimizer"] = "Adam"
parameters["hidden_size"] = 200
parameters["pre_emb"] = "glove"

parameters["w_embed_size"] = 300
# parameters["dim_cap"] = 10

parameters["batch_size"] = 20

parameters["c_embed_size"] = 25
parameters["char_hidden_size"] = 100

parameters["load_embeds"] = True
parameters["dropout"] = 0.2
parameters["gradient_clipping"] = 0
parameters["crf"] = True
parameters["freeze"] = False

epochs = 20
zero_digit = True

assert parameters["pre_emb"] in ["glove", "google"]
assert not parameters["pre_emb"] == "google" or parameters["w_embed_size"] == 300

param_str = "-".join(["%s:%s" % (str(k), str(v)) for (k,v) in parameters.items()]).lower()


In [3]:
model_path = "models/ner/%s/" % param_str
model_path = "models/char5_crf_chars/"

reload = 0

### Load CoNLL03

In [4]:
# CoNLL03
sents_train_03, pos_train_03, chunk_train_03, ner_train_03 = load_conll03(["cleaned_eng.train"])
sents_dev_03, pos_dev_03, chunk_train_03, ner_dev_03 = load_conll03(["cleaned_eng.testa"])
sents_test_03, pos_test_03, chunk_train_03, ner_test_03 = load_conll03(["cleaned_eng.testb"])

print("Train %s, Dev %s, Test %s" % (len(sents_train_03), len(sents_dev_03), len(sents_test_03)))

if zero_digit:
    sents_train_03 = [[zero_digits(w) for w in s] for s in sents_train_03]
    sents_test_03 = [[zero_digits(w) for w in s] for s in sents_test_03]    
    sents_dev_03 = [[zero_digits(w) for w in s] for s in sents_dev_03]    

sents_train = np.concatenate([sents_train_03, sents_dev_03, sents_test_03])

Loaded CoNLL03 in 2.0433363914489746 seconds
Loaded CoNLL03 in 0.5268721580505371 seconds
Loaded CoNLL03 in 0.4434194564819336 seconds
Train 14041, Dev 3250, Test 3453


### Compute or load mappings

In [5]:
if load_mappings("conll03"):
    w2idx, c2idx, ner2idx = load_mappings("conll03")
    idx2w = reverse_dict(w2idx)
    idx2c = reverse_dict(c2idx)
    idx2ner = reverse_dict(ner2idx)
    
else: 
    w2idx, idx2w = word_index(sents_train)
    w2idx, idx2w = add_unknown_last(w2idx, idx2w)

    idner_train, ner2idx, idx2ner = tag_mapping(ner_train_03)
    num_ner_classes = len(ner2idx)

    ner2idx = {v:k for (k,v) in idx2ner.items()}

    c2idx, idx2c = char_index(sents_train)
    c2idx, idx2c = add_unknown_last(c2idx, idx2c)
    save_mappings(w2idx, c2idx, ner2idx, "conll03")

In [6]:
idner_train = tag_mapping(ner_train_03, ner2idx)
idner_dev = tag_mapping(ner_dev_03, ner2idx)
idner_test = tag_mapping(ner_test_03, ner2idx)

### Load pretrained embeddings

In [7]:
if parameters["pre_emb"] == "glove":
    embeddings_path = "word_embeddings/glove.6B/glove.6B.%sd_w2vformat.txt" % parameters["w_embed_size"]
    binary = False
else:
    embeddings_path = "word_embeddings/google/GoogleNews-vectors-negative300.bin"
    binary = True
    
if parameters["load_embeds"]:
    loaded_embeddings, (w2idx_embeds, idx2w_embeds) = load_embeddings(embeddings_path, binary=binary)
else:
    parameters["freeze"] = 0

Loading from saved embeddings
Loading vocab


### Fit word embeddings to vocabulary

In [8]:
char_embeddings = np.random.normal(scale=0.001, size=(len(c2idx), parameters["c_embed_size"]))
embeddings = np.random.normal(scale=0.001, size=(len(w2idx), parameters["w_embed_size"]))

if parameters["load_embeds"]:
    for w, i in w2idx.items():
        idx = w2idx_embeds.get(w.lower())
        if idx is not None:
            embeddings[i] = loaded_embeddings[idx][:parameters["w_embed_size"]]

### Reload model

In [9]:
model = Tagger(model_path, tensor(embeddings),  parameters["hidden_size"], w2idx, c2idx, ner2idx, char_embeddings=tensor(char_embeddings),
               char_hidden_dim = parameters["char_hidden_size"], dropout=parameters["dropout"], 
               crf=parameters["crf"])
if reload:
    model.reload()

### Train

In [10]:
model.train(sents_train_03, sents_dev_03, sents_test_03, idner_train, idner_dev, idner_test,
            parameters["lr"], lr_method=parameters["optimizer"], lr_decay=0.9, 
            batch_size = parameters["batch_size"], eps_noimprov=20, epochs=1000, 
            freeze_embeddings=parameters["freeze"])

  0%|          | 0/703 [00:00<?, ?it/s]

Epoch 1/1000 :


100%|██████████| 163/163 [00:32<00:00,  4.86it/s]


ner : p 0.960430107527, r 0.962952799122, f 0.961689798955, acc 0.9564658697091235


  0%|          | 0/173 [00:00<?, ?it/s]

...

ner : p 0.962378640777, r 0.96081698003, f 0.961597176358, acc 0.9563475826424034
ID     NE  Total      O  B-LOC  B-PER  B-ORG  I-PER  I-ORG B-MISC  I-LOC I-MISC  Percent
 0      O  38323  37688     37     62    115     14    111    189      6    101   98.343
 1  B-LOC   1668     41   1480      7     81      0     15     40      2      2   88.729
 2  B-PER   1617     78     22   1441     40     16      4     12      1      3   89.116
 3  B-ORG   1661    142     68     26   1340      0     35     48      0      2   80.674
 4  I-PER   1156     43      0     45      2   1041     17      3      1      4   90.052
 5  I-ORG    835     63      7      1     46      7    673      2     17     19   80.599
 6 B-MISC    702     75     13     10     28      0      0    571      0      5   81.339
 7  I-LOC    257     15     12      0      2      4     42      0    174      8   67.704
 8 I-MISC    216     41      0      0      0      0     14     11      3    147   68.056
44555/46435 (95.95133%)
NER 

  0%|          | 0/703 [00:00<?, ?it/s]

Epoch 6/1000 :


 18%|█▊        | 125/703 [00:34<02:38,  3.64it/s]

KeyboardInterrupt: 